# LAPACK

интерфейс Python:

https://docs.scipy.org/doc/scipy/reference/linalg.lapack.html

Подробный гайд по LAPACK:

https://netlib.org/lapack/

https://netlib.org/lapack/explore-html/index.html





In [11]:
from scipy.linalg import lapack
import scipy.linalg as la
import numpy as np


In [15]:
n = 100
A = np.array(np.random.randn(n, n), order='F')

def get_Q_qr(A):
    Q, R = la.qr(A, mode='economic')
    return Q

Q = get_Q_qr(A)

print(Q.shape)
print(np.linalg.norm(Q.T @ Q - np.eye(n))) # measure how close Q is to orthogonal

(100, 100)
8.689175407009058e-15


In [16]:
def get_Q_lapack(A):
    qr, tau, work,  info = lapack.dgeqrf(A)
    Q, work, info = lapack.dorgqr(qr, tau)
    return Q

In [17]:
A = np.array(np.random.randn(n, n), order='F')
Q = get_Q_lapack(A)

print(Q.shape)
print(np.linalg.norm(Q.T @ Q - np.eye(n))) # measure how close Q is to orthogonal

(100, 100)
8.184830570651917e-15


In [13]:
print(np.linalg.norm(Q - Q2))

14.142234647957673


Перезаписывать A:

In [18]:
def get_Q_inplace(A):
    m, n = A.shape
    lwork = max(3*n, 1)
    qr, tau, work, info = lapack.dgeqrf(A, lwork, 1) # overwrite A = True
    Q, work, info = lapack.dorgqr(qr, tau, lwork, 1) # overwrite qr = True
    return Q

A = np.array(np.random.randn(n, n), order='F')
Q = get_Q_inplace(A)

print(Q.shape)
print(np.linalg.norm(Q.T @ Q - np.eye(n))) # measure how close Q is to orthogonal
print(np.linalg.norm(A - Q)) # A now contains Q

(100, 100)
8.977688258257193e-15
0.0


Измерим время:

In [20]:
import time

In [45]:
n = 1024
A = np.array(np.random.randn(n, n), order='F')

for get_Q in (get_Q_qr, get_Q_lapack, get_Q_inplace):
    
    print("timing {}".format(get_Q))    
    t0 = time.time()
    Q = get_Q(A)
    t1 = time.time()
    print("  time elapsed = {} sec.".format(t1 - t0))  

timing <function get_Q_qr at 0x0000027808E54160>
  time elapsed = 0.11983633041381836 sec.
timing <function get_Q_lapack at 0x000002780935B700>
  time elapsed = 0.12782835960388184 sec.
timing <function get_Q_inplace at 0x0000027807A36B80>
  time elapsed = 0.10413861274719238 sec.


768
